In [4]:
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
from scipy.spatial.distance import cityblock
from array import array

def evaluateEER(user_scores, imposter_scores, subject):
    countWolf = 0
    countSheep = 0
    countGoat = 0
    countLamb = 0
    labels = [0] * len(user_scores) + [1] * len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    for fp in farates: #if farates high then it means they are more like wolf 
        if fp >= 0.85:
            countWolf += 1
        elif fp >0.45 and fp<=0.85:
            countLamb += 1
        
    for tpr in tpr: #if genuine user are able to login correctly tpr value should be 1
        if tpr >= 0.85:
            countSheep += 1

    for ms in missrates: #Goats refer to users who are prone to false rejects.
        if ms >= 0.85:
            countGoat += 1
    return  subject, countWolf, countSheep, countGoat,countLamb


class ManhattanDetector:

    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects

    def training(self):
        self.mean_vector = self.train.mean().values

    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)

        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, self.mean_vector)
            self.imposter_scores.append(cur_score)

    def evaluate(self):
        eers = []
        # Wolf = []
        self.user_scores = []
        self.imposter_scores = []
        for subject in subjects:
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]

            self.train = genuine_user_data[:200]
            self.test_genuine = genuine_user_data[200:]
            self.test_imposter = imposter_data.groupby("subject").head(10).loc[:, "H.period":"H.Return"]
            self.training()
            self.testing()
            eers.append(evaluateEER(self.user_scores, self.imposter_scores, subject))

        return eers

path = "DSL-StrongPasswordData.csv"
data = pd.read_csv(path)
subjects = data["subject"].unique()
# print("average EER for Manhattan detector:")
array=ManhattanDetector(subjects).evaluate()

dataframe=pd.DataFrame(array)
behaviour_frame=dataframe.rename(columns = {0: 'Subject', 1: 'Wolf',2: 'Sheep', 3: 'Goat', 4: 'Lamb'}) 
behaviour_frame.head(10)

,Subject,Wolf,Sheep,Goat,Lamb
0,s002,39,74,2,70
1,s003,71,156,2,128
2,s004,95,260,2,188
3,s005,105,366,2,208
4,s007,124,469,2,230
5,s008,142,557,2,272
6,s010,88,675,2,340
7,s011,104,769,2,368
8,s012,110,859,2,378
9,s013,93,954,2,430
